In [1]:
import wrds, os
import numpy as np
import pandas as pd

In [2]:
os.chdir('..')

---

In [3]:
db = wrds.Connection()

Enter your WRDS username [maltemax]:mmax_uva
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: n
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [20]:
df = db.get_table(library = 'wrdsapps', table = 'ibcrsphist')

In [24]:
df.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'crsp_ibes_link_raw.tsv'),
          sep = '\t', index = False)

In [46]:
df = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'crsp_ibes_link_raw.tsv'),
                 sep = '\t')

scores:
1. 8-digit historical CUSIP match
2. Historical ticker match + 6-digit CUSIPs and similar company names
3. Historical ticker match + plus 6-digit CUSIPs, but different company names
4. Historical ticker match + similar company names, but different 6-digit CUSIPs
5. Historical ticker match, but both 6-digit CUSIPs and company names are different
6. No matching, or a one-to-many match that cannot be resolved


also: https://wrds-www.wharton.upenn.edu/pages/wrds-research/applications/python-replications/linking-ibes-and-crsp-data-python/

In [47]:
df = df.loc[df['score'] == 1]

In [48]:
df['date_from'], df['date_to'] = pd.to_datetime(df['sdate']), pd.to_datetime(df['edate'])

df['fyear_from'] = np.where(df['date_from'].dt.month < 6,
                            df['date_from'].dt.year,
                            df['date_from'].dt.year + 1)

df['fyear_to'] = np.where(df['date_to'].dt.month < 6,
                          df['date_to'].dt.year - 1,
                          df['date_to'].dt.year)

In [49]:
df = df.loc[~(df['fyear_from'] > df['fyear_to'])].reset_index(drop = True)

In [50]:
df['fyear_to'] = df['fyear_to'].fillna(2024)

In [53]:
df

,ticker,permno,fyear_from,fyear_to
0,0000,14471.0,2014,2016
1,0001,14392.0,2014,2018
2,0004,14418.0,2014,2018
3,000R,14378.0,2014,2019
4,000V,14423.0,2014,2015
...,...,...,...,...
27020,ZXZX,12720.0,2012,2016
27021,ZY,40539.0,1978,1989
27022,ZY,40539.0,1990,2023
27023,ZYNE,15646.0,2016,2023


In [52]:
df = df.drop(columns = ['sdate', 'edate', 'date_from', 'date_to', 'score', 'ncusip'])

In [54]:
between = df.to_dict(orient = 'index')

In [55]:
in_between = []

for k, v in between.items():
    in_between.append([i for i in range(int(v['fyear_from']), int(v['fyear_to']) + 1)])

In [56]:
df['fyear'] = pd.Series(in_between)

In [57]:
df = df.explode('fyear')

In [58]:
df = df.drop(columns = ['fyear_from', 'fyear_to'])

In [60]:
df.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'crsp_ibes_link_final.tsv'),
          sep = '\t', index = False)